In [1]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [0]:

# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])

In [8]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip 
#!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-09-07 18:41:39--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2019-09-07 18:41:39--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2019-09-07 18:41:39--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#df_pages_all = pd.read_pickle("/content/gdrive/My Drive/df_pages_all_backup.pkl")
df_pages_all = pd.read_pickle("/content/gdrive/My Drive/df_pages_all_backup.pkl")

In [0]:
#Filter out nulls
#df = df_pages_all[df_pages_all.initial_message_text == None]
df = df_pages_all[df_pages_all.initial_message_text.notnull() == True]

#df = pd.DataFrame(np.random.randn(100, 2))

# msk = np.random.rand(len(df)) < 0.8

# train = df[msk]

# test = df[~msk]


from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)


In [0]:
docs = train.initial_message_text.to_list()
labels = train.section_number.to_list()

docs_test = test.initial_message_text.to_list()
labels_test = test.section_number.to_list()

In [0]:
import zipfile

#zip_ref = zipfile.ZipFile("glove.6B.zip", 'r')
zip_ref = zipfile.ZipFile("glove.twitter.27B.zip", 'r')
#zip_ref.extractall("/content/gdrive/My Drive/glove.twitter.27B/")
zip_ref.extractall("")
zip_ref.close()


In [0]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
#print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 200
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)

In [7]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/gdrive/My Drive/glove.twitter.27B/glove.twitter.27B.100d.txt')
#f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193514 word vectors.


In [0]:

# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100)) #100
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [29]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=200, trainable=False) #100
model.add(e)
model.add(Flatten())
model.add(Dense(7, activation='sigmoid')) # relu
# compile the model
 
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
#model.compile(optimizer='adadelta', loss='sparse_categorical_crossentropy', metrics=['acc'])
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.compile(optimizer='adam',#keras.optimizers.Adadelta(),
            loss='sparse_categorical_crossentropy',
            metrics=['sparse_categorical_accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 200, 100)          7433300   
_________________________________________________________________
flatten_8 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 7)                 140007    
Total params: 7,573,307
Trainable params: 140,007
Non-trainable params: 7,433,300
_________________________________________________________________
None


In [30]:
# fit the model
model.fit(padded_docs, labels, epochs=13, verbose=1)

Epoch 1/13
32446/32446 [==============================] - 9s 269us/step - loss: 1.4333 - sparse_categorical_accuracy: 0.4493
Epoch 2/13
32446/32446 [==============================] - 7s 211us/step - loss: 1.0543 - sparse_categorical_accuracy: 0.5867
Epoch 3/13
32446/32446 [==============================] - 7s 211us/step - loss: 0.8536 - sparse_categorical_accuracy: 0.6599
Epoch 4/13
32446/32446 [==============================] - 9s 278us/step - loss: 0.7247 - sparse_categorical_accuracy: 0.7082
Epoch 5/13
32446/32446 [==============================] - 9s 291us/step - loss: 0.6376 - sparse_categorical_accuracy: 0.7413
Epoch 6/13
32446/32446 [==============================] - 9s 274us/step - loss: 0.5803 - sparse_categorical_accuracy: 0.7693
Epoch 7/13
32446/32446 [==============================] - 8s 247us/step - loss: 0.5295 - sparse_categorical_accuracy: 0.7892
Epoch 8/13
32446/32446 [==============================] - 8s 250us/step - loss: 0.4910 - sparse_categorical_accuracy: 0.8057


In [31]:
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 88.087900


In [0]:
# integer encode the documents
encoded_docs_test = t.texts_to_sequences(docs_test)
#print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 200
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
#print(padded_docs)

In [0]:
predictions = model.predict(padded_docs_test)

In [112]:
predictions_df = pd.DataFrame(data = predictions,
                              columns = ['0MainPrediction',
                                         '1BackstagePrediction',
                                         '2FOHPrediction',
                                         '3Practice_RoomPrediction',
                                         '4BacklinePrediction',
                                         '5Merch_StandPrediction',
                                         '6BarPrediction']).round(4)
predictions_df['Max'] = predictions_df.idxmax(axis=1)
predictions_df['Actual'] = labels_test
predictions_df['Actual'] = predictions_df['Actual'].astype(str)
predictions_df['pred'] = predictions_df['Max'].astype(str).str[0]
#predictions_df['accurate'] = 0
#predictions_df.accurate[(predictions_df.Actual) == (predictions_df.pred)] = 1
predictions_df['accurate'] = np.where((predictions_df['Actual'])==  predictions_df['pred'], 1, 0)
# for i in range(len(predictions_df)):
#   predictions_df['pred'][i] =  predictions_df.Max[i][:1]
predictions_df.head()

,0MainPrediction,1BackstagePrediction,2FOHPrediction,3Practice_RoomPrediction,4BacklinePrediction,5Merch_StandPrediction,6BarPrediction,Max,Actual,pred,accurate
0,0.0132,0.0,0.0,0.0000,0.0000,0.0041,0.0010,0MainPrediction,0,0,1
1,0.0000,0.0,0.0,0.7257,0.0000,0.0000,0.0000,3Practice_RoomPrediction,3,3,1
2,0.2994,0.0,0.0,0.0037,0.0000,0.0000,0.2811,0MainPrediction,3,0,0
3,0.0000,0.0,0.0,0.0000,0.9604,0.0000,0.0000,4BacklinePrediction,4,4,1
4,0.0001,0.0,0.0,0.0519,0.0177,0.0000,0.1474,6BarPrediction,3,6,0


In [0]:
predictions_df.to_csv("/content/gdrive/My Drive/sneapresults.csv")

In [116]:
predictions_df.accurate.value_counts()

0    4414
1    3698
Name: accurate, dtype: int64

In [0]:
unique, counts = np.unique(predictions, return_counts=True)
dict(zip(unique, counts))